### Download Grammar

In [15]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [16]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [17]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

### Node Types

In [18]:
print(bert_tokenizer.node_types)

['>=', 'unary_operator', 'chevron', 'return_statement', 'pair', 'break', 'lambda_parameters', 'exec', 'case_clause', 'false', 'list', 'global', 'format_expression', 'import', '<>', 'block', 'tuple', 'except', 'for', '[', 'pass', 'format_specifier', '^', 'for_statement', 'typed_default_parameter', 'with_statement', 'case', 'assignment', '<<=', 'assert', 'list_splat_pattern', 'and', '<<', 'if_clause', '//', '%=', '(', 'exec_statement', 'ellipsis', 'if_statement', 'continue_statement', 'keyword_separator', ':', 'parenthesized_list_splat', 'finally', ',', 'nonlocal', 'binary_operator', 'module', 'try_statement', 'conditional_expression', 'with_clause', 'as_pattern', 'import_prefix', '/=', 'else', '__future__', 'list_pattern', '@', 'await', 'import_from_statement', '&=', ';', 'expression_list', 'is', '}', 'not', '->', 'augmented_assignment', 'set', 'wildcard_import', '/', '>', 'async', 'if', '^=', '<', 'or', 'interpolation', 'generator_expression', 'decorated_definition', '**', '**=', 'from

### Encodings

In [19]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
code = "def hello_world(a,b):\n    print('hello world')"
target_node_type = "string"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type))

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def hello_world(a,b):
    print('<mask><mask>')


### Code Prediction

In [20]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

### Evaluation

In [21]:
print(code)
print(masked_code)
print(predictions[0])

def hello_world(a,b):
    print('hello world')
def hello_world(a,b):
    print('<mask><mask>')
def hello_world(a,b):
    print('Hello world')
